In [ ]:
import numpy as np
from shapely.geometry import LineString, Polygon, MultiPolygon
from shapely.ops import unary_union, polygonize
import matplotlib.pyplot as plt
import geopandas as gpd

from shapely.geometry import Point

In [ ]:

def extract_intersection_outline(lanes):

    # Step 1: Extract all boundary line segments
    all_polygons = []
    for lane in lanes:
        all_polygons.append(lane['polygon'])
    
    # Step 2: Merge all boundaries and extract the enclosed polygons
    merged_boundaries = unary_union(all_polygons)

    # Step 3: Generate polygons from the merged lines
    polygons = list(polygonize(merged_boundaries))
    
    if not polygons:
        # If no polygon is formed, use buffer-based approach
        return buffer_based_outline(lanes)
    
    # Step 4: Select the polygon that represents the intersection
    # Usually it's the largest polygon
    if len(polygons) == 1:
        return polygons[0]
    else:
        # Take the largest polygon if there are multiple
        return max(polygons, key=lambda p: p.area)

def buffer_based_outline(lanes):
    """
    Alternative approach using buffer operations when line-based
    polygonization doesn't work well.
    """
    # Create a small buffer around each boundary
    buffer_distance = 0.5  # Adjust based on your data's scale
    buffered_lanes = []
    
    for lane in lanes:
        # Buffer each lane as a whole
        left = lane['left_boundary']
        right = lane['right_boundary']
        
        # Create a lane polygon by buffering both sides and connecting ends
        if left.coords[0] != right.coords[0]:  # Check if start points need connecting
            start_connector = LineString([left.coords[0], right.coords[0]])
            all_parts = [left, right, start_connector]
        else:
            all_parts = [left, right]
            
        if left.coords[-1] != right.coords[-1]:  # Check if end points need connecting
            end_connector = LineString([left.coords[-1], right.coords[-1]])
            all_parts.append(end_connector)
        
        lane_outline = unary_union([line.buffer(buffer_distance) for line in all_parts])
        buffered_lanes.append(lane_outline)
    
    # Merge all lane buffers
    intersection_area = unary_union(buffered_lanes)
    
    # Extract the exterior boundary
    if isinstance(intersection_area, MultiPolygon):
        largest_polygon = max(intersection_area.geoms, key=lambda p: p.area)
        return Polygon(largest_polygon.exterior.coords)
    else:
        return Polygon(intersection_area.exterior.coords)

def is_boundary_part_of_outline(boundary, outline, tolerance=1e-8):
    """
    Determine if a boundary linestring is part of the intersection outline.
    
    Parameters:
    boundary -- A shapely LineString representing a lane boundary
    outline -- A shapely Polygon representing the intersection outline
    tolerance -- Distance tolerance for considering a point on the outline
    
    Returns:
    Boolean indicating if the boundary contributes to the outline
    """
    # Sample points along the boundary
    num_points = min(20, len(boundary.coords))
    sample_indices = np.linspace(0, len(boundary.coords) - 1, num_points).astype(int)
    sample_points = [boundary.coords[i] for i in sample_indices]
    
    # Check if sampled points are on the outline
    outline_boundary = outline.exterior
    points_on_outline = 0
    
    for point in sample_points:
        distance = outline_boundary.distance(Point(point))
        if distance <= tolerance:
            points_on_outline += 1
    
    # If most points are on the outline, consider it part of the outline
    return points_on_outline / num_points > 0.7

def identify_outline_boundaries(lanes, intersection_outline):
    """
    Identify which lane boundaries contribute to the intersection outline.
    
    Parameters:
    lanes -- List of dictionaries with 'left_boundary' and 'right_boundary' keys
    intersection_outline -- Shapely Polygon representing the outline
    
    Returns:
    List of boundary linestrings that form the outline
    """
    
    
    outline_boundaries = []
    
    for i, lane in enumerate(lanes):
        # Check left boundary
        if is_boundary_part_of_outline(lane['left_boundary'], intersection_outline):
            outline_boundaries.append({
                'lane_index': i,
                'boundary_type': 'left',
                'linestring': lane['left_boundary']
            })
        
        # Check right boundary
        if is_boundary_part_of_outline(lane['right_boundary'], intersection_outline):
            outline_boundaries.append({
                'lane_index': i,
                'boundary_type': 'right',
                'linestring': lane['right_boundary']
            })
    
    return outline_boundaries

def visualize_intersection(lanes, intersection_outline=None, outline_boundaries=None):
    """
    Visualize the intersection, its outline, and contributing boundaries.
    """
    plt.figure(figsize=(12, 10))
    
    # Plot all lane boundaries
    for i, lane in enumerate(lanes):
        plt.plot(*lane['left_boundary'].xy, 'b-', alpha=0.5, label='Left boundary' if i == 0 else "")
        plt.plot(*lane['right_boundary'].xy, 'g-', alpha=0.5, label='Right boundary' if i == 0 else "")
    
    # Plot the intersection outline if provided
    if intersection_outline:
        plt.plot(*intersection_outline.exterior.xy, 'r-', linewidth=2, label='Intersection outline')
    
    # Highlight the contributing boundaries if provided
    if outline_boundaries:
        for boundary in outline_boundaries:
            color = 'purple' if boundary['boundary_type'] == 'right' else 'orange'
            plt.plot(*boundary['linestring'].xy, color=color, linewidth=2.5, 
                    label=f"{boundary['boundary_type'].capitalize()} boundary in outline" if boundary == outline_boundaries[0] else "")
    
    plt.axis('equal')
    plt.grid(True)
    plt.legend()
    plt.title('Intersection with Lane Boundaries and Outline')
    plt.show()



In [ ]:
lane_group_df = gpd.read_file("/home/daniel/asim_workspace/asim/notebooks/carla_town05.gpkg", layer="lane_group")
lane_group_df = lane_group_df[~lane_group_df["intersection_id"].isna()]


intersection_ids = list(set(lane_group_df["intersection_id"]))
intersection_id = intersection_ids[2]
intersection_df = lane_group_df[lane_group_df["intersection_id"] == intersection_id]

# fig, ax = plt.subplots(figsize=(10, 10))
# intersection_df.plot(ax=ax)
# intersection_df

print(intersection_id)


left_boundaries = list(intersection_df["left_boundary"])
right_boundaries = list(intersection_df["right_boundary"])
polygons = list(intersection_df["geometry"])
lanes = []
for i in range(len(left_boundaries)):
    lane = {
        'left_boundary': left_boundaries[i],
        'right_boundary': right_boundaries[i],
        "polygon": polygons[i]
    }
    lanes.append(lane)

# lanes = [
#     {
#         'left_boundary': LineString([(0, 0), (10, 0)]),
#         'right_boundary': LineString([(0, 5), (10, 5)])
#     },
#     {
#         'left_boundary': LineString([(10, 0), (20, 0)]),
#         'right_boundary': LineString([(10, 5), (20, 5)])
#     },
#     # Add more lanes as needed
# ]


outline = extract_intersection_outline(lanes)
# outline_boundaries = identify_outline_boundaries(lanes, outline)
# visualize_intersection(lanes, outline, outline_boundaries)

print(isinstance(outline, MultiPolygon))


polygons = list(polygonize(outline))

polygons[0]


In [ ]:
left_boundaries

In [ ]:
lane_group_df = gpd.read_file("/home/daniel/asim_workspace/asim/notebooks/carla_town05.gpkg", layer="intersection")
lane_group_df["lane_group_ids"][0]